# Problema2 - REALIZAR SOBRE ARCHIVO .PY  (NO JUPYTER) 


Lea el archivo 'winemag-data-130k-v2.csv' y realice lo siguiente:

- Explore el dataframe según lo visto en clase
- Realice al menos  4 renombre de columna y 3 creación de nuevas columnas según criterio. Deberá crear las columnas que crea conveniente. 
    Ejemplo: Según el país etiquetelos según continente.

- Genere 4 reportes distintos(podria agrupando, filtrar, ordenar, etc). Deberá elegir que reportes realizar
    - Ejemplo: Por contienente mostrar los vinos mejor puntuados
    - Ejemplo2: Promedio de precio de vino y cantidad de reviews obtenidos según pais. Ordenado de mejor a peor.



- Exporte los 4 reportes a 4 formatos de archivos diversos, csv, excel, sqllite, mongodb...
   
   Para guardar datos de datos agrupados, puede revisar esta solución https://stackoverflow.com/questions/25789264/pandas-dataframegroupby-export-to-excel

- Envie uno de los reportes generados por correo, mostrar un pantallazo del correo enviado




### Datos 

winemag-data-130k-v2.csv
- https://github.com/gdelgador/ProgramacionPython202401/blob/main/Modulo5/src/winemag-data-130k-v2.csv

País según continente:
- https://gist.githubusercontent.com/kintero/7d1db891401f56256c79/raw/a61f6d0dda82c3f04d2e6e76c3870552ef6cf0c6/paises.csv
 






In [1]:
import pandas as pd

# Ruta relativa al archivo existente
url = "./data/winemag-data-130k-v2.csv"

# Cargar el CSV en un DataFrame
df = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(df.head())

   Unnamed: 0   country                                        description  \
0           0     Italy  Aromas include tropical fruit, broom, brimston...   
1           1  Portugal  This is ripe and fruity, a wine that is smooth...   
2           2        US  Tart and snappy, the flavors of lime flesh and...   
3           3        US  Pineapple rind, lemon pith and orange blossom ...   
4           4        US  Much like the regular bottling from 2012, this...   

                          designation  points  price           province  \
0                        Vulkà Bianco      87    NaN  Sicily & Sardinia   
1                            Avidagos      87   15.0              Douro   
2                                 NaN      87   14.0             Oregon   
3                Reserve Late Harvest      87   13.0           Michigan   
4  Vintner's Reserve Wild Child Block      87   65.0             Oregon   

              region_1           region_2         taster_name  \
0              

In [2]:
df.rename(columns={
    'country': 'País',
    'description': 'Descripción',
    'designation': 'Designación',
    'price': 'Precio'
}, inplace=True)

In [3]:
# Crear una columna de Continent basada en el país
continent_mapping = {
    'Italy': 'Europe',
    'Portugal': 'Europe',
    'US': 'North America'
}

df['Continente'] = df['País'].map(continent_mapping)

# Crear una columna de Valoración Alta si el puntaje es 90 o más
df['Valoración Alta'] = df['points'] >= 90

# Crear una columna de Precio en Rango Bajo o Alto
df['Rango de Precio'] = pd.cut(df['Precio'].fillna(0), bins=[0, 20, 50, 100, float('inf')],
                              labels=['Bajo', 'Medio', 'Alto', 'Muy Alto'])


In [4]:
# Crear una columna de Continent basada en el país
continent_mapping = {
    'Italy': 'Europe',
    'Portugal': 'Europe',
    'US': 'North America'
}

df['Continente'] = df['País'].map(continent_mapping)

# Crear una columna de Valoración Alta si el puntaje es 90 o más
df['Valoración Alta'] = df['points'] >= 90

# Crear una columna de Precio en Rango Bajo o Alto
df['Rango de Precio'] = pd.cut(df['Precio'].fillna(0), bins=[0, 20, 50, 100, float('inf')],
                              labels=['Bajo', 'Medio', 'Alto', 'Muy Alto'])


In [5]:
#reportes
# 1.Agrupar por Continente y obtener el vino con la mejor puntuación
top_wines_by_continent = df.loc[df.groupby('Continente')['points'].idxmax()]

# Exportar a CSV
top_wines_by_continent.to_csv('/workspaces/PC5/top_wines_by_continent.csv', index=False)


In [6]:
#reporte
# 2.Agrupar por País y calcular el promedio de precio y la cantidad de reseñas
!pip install openpyxl
price_reviews_by_country = df.groupby('País').agg({
    'Precio': 'mean',
    'title': 'count'
}).rename(columns={'title': 'Cantidad de Reviews'})

# Exportar a Excel
price_reviews_by_country.to_excel('/workspaces/PC5/price_reviews_by_country.xlsx')



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [7]:
# 3.Distribución de Puntuaciones por Región
# Agrupar por Región y obtener estadísticas de puntuación
score_distribution_by_region = df.groupby('province')['points'].describe()

# Exportar a SQLite
import sqlite3

conn = sqlite3.connect('/workspaces/PC5/score_distribution.db')
score_distribution_by_region.to_sql('score_distribution', conn, if_exists='replace')
conn.close()
